# Backpropagation

## References

* _Rumelhart et al. 1986, Learning representations by back-propagating errors_, [nature](https://www.nature.com/articles/323533a0)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn.datasets as sk_datasets
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
from torch.utils.data import DataLoader

import random_neural_net_models.backprop_rumelhart as backprop_rumelhart
import random_neural_net_models.data as rnnm_data
import random_neural_net_models.learner as rnnm_learner

In [ ]:
SEED = 42

In [ ]:
X, y = sk_datasets.make_regression(
    n_samples=100, n_features=1, noise=20, random_state=SEED
)

In [ ]:
sns.scatterplot(x=X[:, 0], y=y)
plt.tight_layout()

In [ ]:
X, y = sk_datasets.make_blobs(
    n_samples=1_000,
    n_features=2,
    centers=2,
    random_state=SEED,
)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, alpha=0.3);

## Rumelhart et al. 1986, "Learning representations by back-propagating errors"
> The following is in the spirit of the paper, *cough*. Found the presentation in the paper somewhat hard to read.

With input $x$ we predict $y$ using a function $f$ like

$$

y = f(x)

$$

$f$ uses weight $w$ and some non-linearity $g$ like

$$
z =  x \cdot w^T \\
f(x) = g(z)
$$

In the paper they use the sigmoid
$$
g(z) = \frac{1}{1+\exp(-z)}
$$

Indicating repeating weight multiplication and non-linearity with index $i$ (layer), we can write

$$

z_1 =  x \cdot w_1 ^ T \\
a_1 = g_1(z_1) \\

...\\

z_{i} = a_{i-1} \cdot w_{i}^T \\
a_{i} = g_{i}(z_{i}) \\

...\\

z_N = a_{N-1} \cdot w_N^T \\
a_N = g_N(z_N) \\

f(x) = a_N

$$

where $a_i$ is "activation" of layer $i$

Comparing predictions to desired valued $d$ we denote deviations / the loss as $l(d,y)$.

With this we can identify how to change $w_i$ by how much lead to the strongest improvement of $l(d,y)$ for any $x$, $y$, $d$ by differentiating $l$. A straightforward way is scaling that gradient and applying it like

$$

w_{i,\text{new}} = w_{i,\text{old}} + \epsilon \frac{d}{dw_i}l(d,y)

$$

with some factor $\epsilon \in [0,1]$

$\frac{d}{dw_i}l(d,y)$ can analytically be derived using the chain rule as 

$$

\frac{d}{dw_i}l(d,y) = l^\prime(d,y) \cdot g_N^\prime(z_N) \cdot z_N^\prime \cdot g_{N-1}^\prime(z_{N-1}) \cdot z_{N-1}^\prime \cdot \space ... \space \cdot g_{i}^\prime(z_{i}) \cdot a_{i-1}

$$

Replacing $z^\prime = \frac{\partial}{\partial a}z = w ^ T$
$$

\frac{d}{dw_i}l(d,y) = l^\prime(d,y) \cdot g_N^\prime(z_N) \cdot w_N ^ T \cdot g_{N-1}^\prime(z_{N-1}) \cdot w_{N-1} ^ T \cdot \space ... \space \cdot g_{i}^\prime(z_{i}) \cdot a_{i-1}

$$

Another version proposed in Rumelhart et al. is
$$

w_{i,\text{new}} = w_{i,\text{old}} + \epsilon \frac{d}{dw_i}l(d,y) + \alpha \left( \frac{d}{dw_i}l(d,y) \right)_\text{old}

$$

with some factor $\alpha \in [0,1]$.

In [ ]:
x = torch.linspace(-10, 10, 100)
a = backprop_rumelhart.sigmoid(x)
a_prime = backprop_rumelhart.sigmoid_derivative(x)

In [ ]:
sns.lineplot(x=x, y=a, label="g(z): sigmoid")
sns.lineplot(x=x, y=a_prime, label="dg(z)/dz: sigmoid derivative")

In [ ]:
model = backprop_rumelhart.Rumelhart1986PerceptronClassifier(
    n_hidden=(10, 5), epochs=10, verbose=True, eps=1e-3, alpha=1e-3
)

In [ ]:
model.fit(X, y)

In [ ]:
model.errors_

In [ ]:
x0 = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)
x1 = np.linspace(X[:, 1].min(), X[:, 1].max(), 100)
X0, X1 = np.meshgrid(x0, x1)
X_plot = np.array([X0.ravel(), X1.ravel()]).T
X_plot[:4]

In [ ]:
y_prob = model.predict_proba(X_plot)

In [ ]:
fig, ax = plt.subplots()
im = ax.pcolormesh(X0, X1, y_prob.reshape(X0.shape), alpha=0.2)
fig.colorbar(im, ax=ax)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, ax=ax, alpha=0.3)
plt.show()

In [ ]:
y_pred = model.predict(X)

In [ ]:
print(metrics.classification_report(y, y_pred))

## and now with `Learner` & pytorch tools

In [ ]:
ds = rnnm_data.TabularNumpyDatasetTrain(X, y)

In [ ]:
ds[0]

In [ ]:
dl = DataLoader(
    ds,
    batch_size=10,
    collate_fn=rnnm_data.tabular_numpy_collate_train,
    shuffle=True,
)

In [ ]:
model = backprop_rumelhart.Rumelhart1986PytorchPerceptron(
    n_hidden=(2, 10, 5, 1)
)

In [ ]:
lr_find_callback = rnnm_learner.LRFinderCallback(1e-5, 100, 100)

loss_callback = rnnm_learner.TrainLossCallback()
activations_callback = rnnm_learner.TrainActivationsCallback(
    every_n=10, max_depth_search=4, name_patterns=(".*act",)
)
gradients_callback = rnnm_learner.TrainGradientsCallback(
    every_n=10, max_depth_search=4, name_patterns=(".*lin",)
)
parameters_callback = rnnm_learner.TrainParametersCallback(
    every_n=10, max_depth_search=4, name_patterns=(".*lin",)
)
callbacks = [
    loss_callback,
    activations_callback,
    gradients_callback,
    parameters_callback,
]

optimizer = optim.SGD(model.parameters(), lr=10, momentum=1e-3)
loss = backprop_rumelhart.BCELoss()
save_dir = Path("./models")
learner = rnnm_learner.Learner(
    model, optimizer, loss, callbacks=callbacks, save_dir=save_dir
)

In [ ]:
learner.find_learning_rate(dl, n_epochs=10, lr_find_callback=lr_find_callback)

In [ ]:
lr_find_data = lr_find_callback.get_losses()
lr_find_data.head(2)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

sns.lineplot(data=lr_find_data, x="lr", y="smooth_loss", ax=ax, label="smooth")
sns.lineplot(data=lr_find_data, x="lr", y="loss", ax=ax, label="raw")
ax.legend(title="loss type")
plt.tight_layout()

In [ ]:
learner.fit(dl, n_epochs=20)

In [ ]:
losses = loss_callback.get_losses()
sns.scatterplot(data=losses, x="iteration", y="loss")

In [ ]:
parameters = parameters_callback.get_stats()
parameters.head(2)

In [ ]:
fig, axs = plt.subplots(figsize=(12, 12), nrows=3, sharex=True)

ax = axs[0]
sns.scatterplot(data=parameters, x="iter", y="mean", hue="name", ax=ax)

ax = axs[1]
sns.scatterplot(data=parameters, x="iter", y="std", hue="name", ax=ax)

ax = axs[2]
sns.scatterplot(data=parameters, x="iter", y="abs_perc90", hue="name", ax=ax)

plt.tight_layout()

In [ ]:
gradients = gradients_callback.get_stats()
gradients.head(2)

In [ ]:
fig, axs = plt.subplots(figsize=(12, 12), nrows=4, sharex=True)

ax = axs[0]
sns.scatterplot(data=gradients, x="call", y="mean", hue="name", ax=ax)

ax = axs[1]
sns.scatterplot(data=gradients, x="call", y="std", hue="name", ax=ax)

ax = axs[2]
sns.scatterplot(data=gradients, x="call", y="frac_dead", hue="name", ax=ax)

ax = axs[3]
sns.scatterplot(data=gradients, x="call", y="abs_perc90", hue="name", ax=ax)

plt.tight_layout()

In [ ]:
activations = activations_callback.get_stats()
activations.head(2)

In [ ]:
fig, axs = plt.subplots(figsize=(12, 12), nrows=3, sharex=True)

ax = axs[0]
sns.scatterplot(data=activations, x="call", y="mean", hue="name", ax=ax)

ax = axs[1]
sns.scatterplot(data=activations, x="call", y="std", hue="name", ax=ax)

ax = axs[2]
sns.scatterplot(data=activations, x="call", y="frac_dead", hue="name", ax=ax)

plt.tight_layout()

In [ ]:
x0 = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)
x1 = np.linspace(X[:, 1].min(), X[:, 1].max(), 100)
X0, X1 = np.meshgrid(x0, x1)
X_plot = np.array([X0.ravel(), X1.ravel()]).T
X_plot[:4]

In [ ]:
ds_plot = rnnm_data.TabularNumpyDatasetEval(X_plot)
dl_plot = DataLoader(
    ds_plot, batch_size=5, collate_fn=rnnm_data.tabular_numpy_collate_eval
)

In [ ]:
y_prob = learner.predict(dl_plot)

In [ ]:
y_prob = y_prob.detach().numpy()
y_prob

In [ ]:
fig, ax = plt.subplots()
im = ax.pcolormesh(X0, X1, y_prob.reshape(X0.shape), alpha=0.2)
fig.colorbar(im, ax=ax)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, ax=ax, alpha=0.3)
plt.show()